<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 20px; font-family: TimesNewRoman; color: skyblue">
    TIME SERIES DATA PROCESSING
    <br>
    RESERVOIRS LEVEL
<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
    Main Formatting Notebook
</div>
</div>
<br>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Each part of the following script was used to proccess the raw data for the Reservoirs Level Time Series Raw Data for all the european countries of the Dispa-SET_Unleash project.
<br>
Read explanation text cells to follow and understand all the process until final results were got stept by step.
</div>
<br>
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    1. Notebook Set Up
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Importing needed libraries
</div>

In [1]:
import pandas as pd
import re
import csv
import os
import requests
from urllib.parse import urlparse, parse_qs
import datetime
from bs4 import BeautifulSoup
import http.client
from multiprocessing import Pool
import shutil
import numpy as np
from datetime import datetime

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    2. Dispa-SET_Unleash Folder Path
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Determinning dynamically the zone_folder_path based on the location of the "Dispa-SET_Unleash" folder relative to the current working directory.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
If the "Dispa-SET_Unleash" folder is copied to a different machine or location, the dispaSET_unleash_folder_path variable will automatically adjust accordingly.
</div>

In [2]:
# Get the current working directory
current_directory = os.getcwd()

# Navigate to the parent directory of "Dispa-SET_Unleash"
dispaSET_unleash_parent_directory = os.path.dirname(current_directory)

# Get the path to the "Dispa-SET_Unleash" folder
dispaSET_unleash_folder_path = os.path.dirname(dispaSET_unleash_parent_directory)

# Construct the dispaSET_unleash_folder_name variable
dispaSET_unleash_folder_name = os.path.basename(dispaSET_unleash_folder_path)

print("dispaSET_unleash_folder_name:", dispaSET_unleash_folder_name)
print("dispaSET_unleash_folder_path:", dispaSET_unleash_folder_path)

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
3. Usefull Variable Definition
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Entering a value to all the variables which content are going to be used in some of the next stages of this script. 
</div>
<div style="text-align: left; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Indicate the year of all data is referring to in the variable data_year.
<br>
The universal_standar_time variable is going to be used to download all the time series data in this horary zone. Additionally as each european country belongs a particular time sector the corresponding time series data related to its time sector are going to be downloaded as well but in a different file.
</div>

In [3]:
# Year to which data refers to:
data_year = 2023

# Define lists of countries and standard times
countries = [
    "Austria", "Belgium", "Bulgaria", "Switzerland", "Cyprus", "Czech Republic",
    "Germany", "Denmark", "Estonia", "Greece", "Spain", "Finland", "France",
    "Croatia", "Hungary", "Ireland", "Italy", "Lithuania", "Luxembourg", "Latvia",
    "Malta", "Netherlands", "Norway", "Poland", "Portugal", "Romania", "Sweden",
    "Slovenia", "Slovakia", "United Kingdom"
]

dispaSET_codes = ["AT", "BE", "BG", "CH", "CY", "CZ", "DE", "DK", "EE", "EL", "ES", "FI", "FR", "HR", "HU", 
                  "IE", "IT", "LT", "LU", "LV", "MT", "NL", "NO", "PL", "PT", "RO", "SE", "SI", "SK", "UK"
]

# Universal standad time:
standard_time = 'UTC'

# Western European Time:
#standard_time = 'WET_WEST'

# Central European Time:
#standard_time = 'CET_CEST'

# Eastern European Time:
#standard_time = 'EET_EST'

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
4. Reservoirs Level Directories Definition
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Creating the folders that are going to content all the data realted to the Reservoirs Level time series.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Futher the downloaded raw data is going to be used to get the Reservoirs Level time series. 
</div>

In [4]:
# Additional string to be appended
additional_path = "/RawData/HydroData/ReservoirLevel/"
additional_path_1 = "/RawData/HydroData/ScaledInflows/"
additional_path_2 = "/RawData/HydroData/ReservoirLevel/Time_Series_Raw_Data_Source/"

# Construct the standard_time_data_folder_path variable
reference_data_folder_path = dispaSET_unleash_folder_path + additional_path

# Construct the Reservoir_Level_folder_path variable
reservoir_level_folder_path = dispaSET_unleash_folder_path + additional_path

# Construct the Scalled_Inflow_folder_path variable
scaled_inflows_folder_path = dispaSET_unleash_folder_path + additional_path_1

# Construct the Reservoir_Level_raw_rada_folder_path variable
reservoir_level_raw_data_folder_path = dispaSET_unleash_folder_path + additional_path_2

print("reference_data_folder_path:", reference_data_folder_path)
print("reservoir_level_folder_path:", reservoir_level_folder_path)
print("scaled_inflows_folder_path:", scaled_inflows_folder_path)
print("reservoir_level_raw_data_folder_path:", reservoir_level_raw_data_folder_path)

reference_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/
reservoir_level_folder_path: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/
scaled_inflows_folder_path: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/
reservoir_level_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Series_Raw_Data_Source/


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
    4.1. Back Up Directory
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Saving the original files into a Back up folder.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Since in the next steps of the processing data new subfolders and files are going to be created, the original ones are saved in a back up foldet to return them as its default content ones the process will be finished.
</div>

In [5]:
additional_path_5 = '/RawData/HydroData/ReservoirLevel_backup/'

# Construct the backup_folder_path variable
backup_folder_path = dispaSET_unleash_folder_path + additional_path_5

print("backup_folder_path:", backup_folder_path)

# Create a backup of the directory
if os.path.exists(backup_folder_path):
    shutil.rmtree(backup_folder_path)  # Remove any existing backup if necessary
shutil.copytree(reservoir_level_folder_path, backup_folder_path)

print(f"Backup created at {backup_folder_path}")

backup_folder_path: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel_backup/
Backup created at /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel_backup/


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
5. Reference File
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Creating a file where auxiliar data needed to the fortmatting process is going to be written such as file is named:
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Reference_Data.csv
</div>

In [6]:
# Create DataFrame
df = pd.DataFrame({'Country': countries, 'Dispa-SET_Code': dispaSET_codes, 'Standar_Time': standard_time, 'Data_Year': data_year})


reference_data_file_name = 'Reference_Data.csv'

# Construct the full file path
reference_data_file_path = os.path.join(reference_data_folder_path, reference_data_file_name)

# Create the CSV file with the specified name
with open(reference_data_file_path, 'w') as f:
    # Optional: Write a header if needed
    # f.write("header1,header2,header3\n")

# Save DataFrame to the CSV file
    df.to_csv(reference_data_file_path, index=False)

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
5.1. Folder Data Creation
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Creating the folder by country that is going to storage the processed reservoiur time series.
</div>

In [7]:
# Read the reference data file
df = pd.read_csv(reference_data_file_path)

# Create a new column 'Zone_Folder_Path' and 'Single_Raw_Data_File_Path'
df['Zone_Folder_Path'] = reference_data_folder_path + df['Dispa-SET_Code'].astype(str)
df['Single_Raw_Data_File_Path'] = ""  # Initialize the new column with empty strings

# Iterate over each row and create a CSV file
for index, row in df.iterrows():
    zone_folder_path = row['Zone_Folder_Path']
    output_csv_file_path = os.path.join(zone_folder_path, f"{data_year}_1.csv")
    
    # Create directories if they don't exist
    os.makedirs(os.path.dirname(output_csv_file_path), exist_ok=True)
    
    # Write the CSV file
    with open(output_csv_file_path, 'w') as f:
        # Write header if needed
        f.write("Header1,")
        
        # Write data (if needed)
        # f.write("Data1,Data2,...\n")
        
        # Here, you can write any data you want into the CSV file
    
    # Print the message for file creation
    print(f"CSV file created: {output_csv_file_path}")
    
    # Update the 'Single_Raw_Data_File_Path' column with the path of the created CSV file
    df.at[index, 'Single_Raw_Data_File_Path'] = output_csv_file_path

# Save the updated DataFrame back to the reference data CSV file
df.to_csv(reference_data_file_path, index=False)

CSV file created: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/2023_1.csv
CSV file created: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/2023_1.csv
CSV file created: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/2023_1.csv
CSV file created: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/2023_1.csv
CSV file created: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/2023_1.csv
CSV file created: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CZ/2023_1.csv
CSV file created: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DE/2023_1.csv
CSV file created: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DK/2023_1.csv
CSV file created: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EE/2023_1.csv
CSV file created: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EL/2023_1.csv
CSV file created: /home/ray/Dispa-SET_Unleash/RawData/HydroData/Reserv

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [8]:
print (f"dispaSET_unleash_folder_name:                              {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                              {dispaSET_unleash_folder_path}")
print (f"data_year:                                                 {data_year}")
print (f"reference_data_folder_path:                                {reference_data_folder_path}")
print (f"reference_data_file_name:                                  {reference_data_file_name}")
print (f"reference_data_file_path:                                  {reference_data_file_path}")
print (f"reservoir_level_folder_path:                               {reservoir_level_folder_path}")
print (f"reservoir_level_raw_data_folder_path:                      {reservoir_level_raw_data_folder_path}")
print (f"standard_time:                                             {standard_time}")

dispaSET_unleash_folder_name:                              Dispa-SET_Unleash
dispaSET_unleash_folder_path:                              /home/ray/Dispa-SET_Unleash
data_year:                                                 2023
reference_data_folder_path:                                /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/
reference_data_file_name:                                  Reference_Data.csv
reference_data_file_path:                                  /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Reference_Data.csv
reservoir_level_folder_path:                               /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/
reservoir_level_raw_data_folder_path:                      /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Series_Raw_Data_Source/
standard_time:                                             UTC


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
6. Raw Data
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Giving the path to the reservoir level raw data files.
</div>

In [9]:
def copy_and_update_csv(reference_data_file_path, reservoir_level_raw_data_folder_path, dispaSET_codes):
    """Copies CSV files to new folders and updates the reference data.

    Args:
        reference_data_file_path (str): Path to the reference data CSV file.
        reservoir_level_raw_data_folder_path (str): Path to the raw data folder.
        dispaSET_codes (list): List of country codes.
    """

    df = pd.read_csv(reference_data_file_path)

    for country in dispaSET_codes:
        source_folder = os.path.join(reservoir_level_raw_data_folder_path, country)
        source_file = os.path.join(source_folder, "Raw_Data.csv")
        destination_folder = os.path.join(os.path.dirname(reference_data_file_path), country)
        destination_file = os.path.join(destination_folder, "Raw_Data.csv")

        if not os.path.exists(os.path.dirname(destination_file)):
            os.makedirs(os.path.dirname(destination_file))

        shutil.copy2(source_file, destination_file)
        df.loc[df['Dispa-SET_Code'] == country, 'Raw_Data_File_Path'] = destination_file
        print(f"Copied file: {source_file} to {destination_file}")

    df.to_csv(reference_data_file_path, index=False)

copy_and_update_csv(reference_data_file_path, reservoir_level_raw_data_folder_path, dispaSET_codes)

Copied file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Series_Raw_Data_Source/AT/Raw_Data.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/Raw_Data.csv
Copied file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Series_Raw_Data_Source/BE/Raw_Data.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/Raw_Data.csv
Copied file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Series_Raw_Data_Source/BG/Raw_Data.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/Raw_Data.csv
Copied file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Series_Raw_Data_Source/CH/Raw_Data.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/Raw_Data.csv
Copied file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Series_Raw_Data_Source/CY/Raw_Data.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/Raw_Data.csv
Copied fil

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
6.1. Raw Data Headers
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Adding column names.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
All the raw data were downloaded without columns identification, the corresponding header to each column is added.
<br>
Additionally an extra row is going to be added to the recently downloaded data for future interpolation processes.
</div>

In [10]:
# Define the path to the CSV file containing the file paths
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"

# Read the CSV file containing the paths
df = pd.read_csv(reference_data_file_path)

success = True

# Iterate through each file path in the column 'Time_Series_Raw_Data_File_Path'
for index, row in df.iterrows():
    file_path = row['Raw_Data_File_Path']
    
    # Open the existing CSV file in read mode
    with open(file_path, 'r', newline='') as file:
        # Read the existing content
        reader = csv.reader(file)
        rows = list(reader)

    # Insert a new empty row at the beginning
    rows.insert(0, [])

    # Write the updated content back to the CSV file
    with open(file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(rows)
        
    print(f"Empty row added at the beginning of file '{file_path}'.")

print("All empty rows added successfully.")

Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/Raw_Data.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/Raw_Data.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/Raw_Data.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/Raw_Data.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/Raw_Data.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CZ/Raw_Data.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DE/Raw_Data.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DK/Raw_Data.csv'.
Empty row added at the beginning

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Giving the Year Columns to the Raw Data.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
The raw data has data from various years in each columns (starting the second column).
<br>
It is needed to identify what year each data belongs to.
<br>
For this, the corresponding header is going to be given to each column.
</div>
<div style="text-align: justify; margin-left: 4.0em; font-weight: unbold; font-size: 12px; font-family: TimesNewRoman; color:skyblue">
The value of the start_year and end_year variables has to be the same than those used in the Reservorius Level Downloading Notebook.
<br>
In the current code, this values are set in 2016 and 2024 since this are the years which are been working with.
</div>

In [11]:
start_year = 2016
end_year = 2024

In [12]:
year_range = list(range(start_year+1, end_year+1))

# Convert the integer values to strings
start_year_str = str(start_year)
year_range_str = [str(year) for year in year_range]

# Create the headers list
headers = ['Week', start_year_str] + year_range_str

# Define the path to the CSV file containing the file paths
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"

# Read the CSV file containing the paths
df = pd.read_csv(reference_data_file_path)

success = True

# Iterate through each file path in the column 'Time_Series_Raw_Data_File_Path'
for index, row in df.iterrows():
    file_path = row['Raw_Data_File_Path']
    
    # Check if the file exists
    if os.path.exists(file_path):
        # Open the file for reading
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Find the index of the first empty row
        empty_row_index = next((i for i, line in enumerate(lines) if line.strip() == ""), None)

        # If an empty row is found, copy the headers to it
        if empty_row_index is not None:
            lines[empty_row_index] = ','.join(headers) + '\n'

            # Write the updated lines back to the file
            with open(file_path, 'w') as file:
                file.writelines(lines)
        else:
            print(f"No empty row found in file '{file_path}'. Headers not copied.")
    else:
        success = False
        print(f"Error: File '{file_path}' does not exist.")

if success:
    print("Headers copied successfully to the first empty row of all files.")
else:
    print("Some errors occurred while copying headers.")

Headers copied successfully to the first empty row of all files.


<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Filling missing weeks
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Some years have 1 week more than others.
<br>
This gaps are filled with the data of subsequents years in each column.
</div>

In [13]:
# Read the reference data CSV file
#reference_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Reference_Data.csv"
df_reference = pd.read_csv(reference_data_file_path)

# Iterate through each zone and its folder path
for index, row in df_reference.iterrows():
    raw_data_file_path = row["Raw_Data_File_Path"]

    # Read the zone's CSV data (handle potential file not found)
    if os.path.exists(raw_data_file_path):
        df = pd.read_csv(raw_data_file_path)
    else:
        print(f"File not found: {raw_data_file_path}")
        continue  # Skip to the next zone if file is missing

    # Process the data (add new row, fill missing values)
    df.loc[len(df)] = ['Week 54'] + [None] * (len(df.columns) - 1)
    for col in df.columns[1:]:
        penultimate_value = df.iloc[-2][col]
        last_value = df.iloc[-1][col]

        if pd.isna(penultimate_value):
            next_col_index = df.columns.get_loc(col) + 1
            if next_col_index < len(df.columns):
                next_col_name = df.columns[next_col_index]
                next_col_first_value = df.iloc[0][next_col_name]
                df.iloc[-2, df.columns.get_loc(col)] = next_col_first_value

        if pd.isna(last_value):
            next_col_index = df.columns.get_loc(col) + 1
            if next_col_index < len(df.columns):
                next_col_name = df.columns[next_col_index]
                next_col_second_value = df.iloc[1][next_col_name]
                df.iloc[-1, df.columns.get_loc(col)] = next_col_second_value

    # Save the modified DataFrame back to the CSV file
    df.to_csv(raw_data_file_path, index=False)

    print(f"Processing completed successfully for file: {raw_data_file_path}")

print("Overall processing finished.")

Processing completed successfully for file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/Raw_Data.csv
Processing completed successfully for file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/Raw_Data.csv
Processing completed successfully for file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/Raw_Data.csv
Processing completed successfully for file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/Raw_Data.csv
Processing completed successfully for file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/Raw_Data.csv
Processing completed successfully for file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CZ/Raw_Data.csv
Processing completed successfully for file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DE/Raw_Data.csv
Processing completed successfully for file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DK/Raw_Data.csv
Processing completed successfull

/tmp/ipykernel_69186/3084917098.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[len(df)] = ['Week 54'] + [None] * (len(df.columns) - 1)
/tmp/ipykernel_69186/3084917098.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1537771' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[-2, df.columns.get_loc(col)] = next_col_first_value
/tmp/ipykernel_69186/3084917098.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior,

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Filling empty N/A or n/e fields
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Some years have no data in their columns so this are replaced by '0'.
</div>

In [14]:
# Read the reference data CSV file
reference_data = pd.read_csv(reference_data_file_path)

# Iterate through each row in the "Raw_Data_File_Path" column
for index, row in reference_data.iterrows():
    raw_data_file_path = row["Raw_Data_File_Path"]

    # Read the raw data CSV file
    raw_data = pd.read_csv(raw_data_file_path)

    # Select columns from the second column onwards
    columns_to_process = raw_data.columns[1:]

    # Replace empty values, 'N/A', and 'n/e' with '0.0' in the selected columns
    before_replace = raw_data[columns_to_process].isnull().sum().sum() + raw_data[columns_to_process].isin(["N/A", "n/e"]).sum().sum()
    raw_data[columns_to_process] = raw_data[columns_to_process].fillna(0.0)
    raw_data[columns_to_process] = raw_data[columns_to_process].replace("N/A", 0.0)
    raw_data[columns_to_process] = raw_data[columns_to_process].replace("n/e", 0.0)
    after_replace = raw_data[columns_to_process].isnull().sum().sum() + raw_data[columns_to_process].isin(["N/A", "n/e"]).sum().sum()
    replaced_values = before_replace - after_replace

    # Process the cleaned raw data as needed
    # ...

    # Save the modified raw data to the original CSV file
    raw_data.to_csv(raw_data_file_path, index=False)

    # Print a message indicating the processed file and replaced values
    print(f"Processed CSV file: {raw_data_file_path}")
    print(f"Replaced {replaced_values} values.")

Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/Raw_Data.csv
Replaced 2 values.
Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/Raw_Data.csv
Replaced 486 values.
Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/Raw_Data.csv
Replaced 4 values.
Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/Raw_Data.csv
Replaced 2 values.
Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/Raw_Data.csv
Replaced 465 values.
Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CZ/Raw_Data.csv
Replaced 486 values.
Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DE/Raw_Data.csv
Replaced 486 values.
Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DK/Raw_Data.csv
Replaced 486 values.
Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroD

/tmp/ipykernel_69186/1241220224.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  raw_data[columns_to_process] = raw_data[columns_to_process].replace("n/e", 0.0)
/tmp/ipykernel_69186/1241220224.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  raw_data[columns_to_process] = raw_data[columns_to_process].replace("n/e", 0.0)
/tmp/ipykernel_69186/1241220224.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objec

Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EL/Raw_Data.csv
Replaced 99 values.
Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/ES/Raw_Data.csv
Replaced 2 values.
Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/FI/Raw_Data.csv
Replaced 3 values.
Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/FR/Raw_Data.csv
Replaced 4 values.
Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/HR/Raw_Data.csv
Replaced 2 values.
Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/HU/Raw_Data.csv
Replaced 486 values.
Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/IE/Raw_Data.csv
Replaced 486 values.
Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/IT/Raw_Data.csv
Replaced 3 values.
Processed CSV file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/R

/tmp/ipykernel_69186/1241220224.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  raw_data[columns_to_process] = raw_data[columns_to_process].replace("n/e", 0.0)
/tmp/ipykernel_69186/1241220224.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  raw_data[columns_to_process] = raw_data[columns_to_process].replace("n/e", 0.0)
/tmp/ipykernel_69186/1241220224.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objec

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Separating the raw data by interested year.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
The raw data content reservoir level information of many years (from the 'start_year' variable to the 'end_year' variable).
<br>
The data belonged to the year defined by the 'data_year' variable is separated to be processed and are saved in a csv file named as the 'data_year' variable plus '_1.csv' e.g. 2023_1.csv
</div>

In [15]:
# Read the CSV file containing the file paths
#reference_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Reference_Data.csv"
df = pd.read_csv(reference_data_file_path)

# Iterate over each file path in the 'Raw_Data_File_Path' column
for file_path in df['Raw_Data_File_Path']:
    # Check if the file exists
    if os.path.exists(file_path):
        # Define the new file name
        new_file_name = 'Raw_Data.csv'
        new_file_path = os.path.join(os.path.dirname(file_path), new_file_name)

        # Rename the original file
        os.rename(file_path, new_file_path)

        # Read the original CSV file
        df_original = pd.read_csv(new_file_path)

        # Extract the first column and the column with the same value as data_year
        new_df = df_original[['Week', str(data_year)]]

        # Define the new file name for the extracted data
        new_file_name_extracted = f"{data_year}_1.csv"
        new_file_path_extracted = os.path.join(os.path.dirname(file_path), new_file_name_extracted)

        # Save the extracted data to a new CSV file
        new_df.to_csv(new_file_path_extracted, index=False)

        print(f"Data extracted and saved to {new_file_path_extracted}")

    else:
        print(f"File '{file_path}' does not exist.")

Data extracted and saved to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/2023_1.csv
Data extracted and saved to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/2023_1.csv
Data extracted and saved to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/2023_1.csv
Data extracted and saved to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/2023_1.csv
Data extracted and saved to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/2023_1.csv
Data extracted and saved to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CZ/2023_1.csv
Data extracted and saved to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DE/2023_1.csv
Data extracted and saved to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DK/2023_1.csv
Data extracted and saved to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EE/2023_1.csv
Data extracted and saved to /home/ray/Dispa-SET_Unleash/RawData/HydroData

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Adding columns to the reference_data file that define the path to the final processed time series reservoir files according its time step.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
This columns are needed in the next stages of the code
</div>

In [16]:
# Read the reference data CSV file
df = pd.read_csv(reference_data_file_path)

# Add new headers
new_headers = ['1h', '30min', '15min']
df = pd.concat([df, pd.DataFrame(columns=new_headers)], axis=1)

# Save the updated DataFrame back to the CSV file
df.to_csv(reference_data_file_path, index=False)

print("New headers added to the reference data file.")

New headers added to the reference data file.


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
6.2. Reservoir Level Factor
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Getting the corresponding reservoir level factor to the extracted raw data files.
<br>
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
As the full storage capacity of each HPSP and HDAM unit it is not provided by the web source. 
<br>
It is going to work with the maximum agregated Storage Energy Value of the corresponding year i.e. defined in the data_year variable.
</div>

In [17]:
def normalize_column(data_year, reference_data_file_path):
    """Normalize the specified column in each CSV file by dividing by the maximum value.
       The maximum value is saved in a new column named 'Maximum_Level' in the reference DataFrame.
       If the maximum value is 0 or the column contains non-numeric values, it will be filled with zeros."""
    
    # Read the reference data file
    df_reference = pd.read_csv(reference_data_file_path)
    
    for index, row in df_reference.iterrows():
        # Construct the path to the folder and the corresponding CSV file
        folder_path = row['Zone_Folder_Path']
        csv_file_path = os.path.join(folder_path, f"{data_year}_1.csv")
        
        # Check if the CSV file exists
        if os.path.exists(csv_file_path):
            # Read the CSV file
            df = pd.read_csv(csv_file_path)
            
            if str(data_year) in df.columns:
                # Convert the column to numeric, replacing non-numeric values with NaN
                df[str(data_year)] = pd.to_numeric(df[str(data_year)], errors='coerce')
                
                # Find the maximum value in the column, ignoring NaN values
                max_value = df[str(data_year)].max()
                
                if pd.notna(max_value) and max_value != 0:  # Avoid division by zero
                    # Normalize the column by dividing each value by the maximum value
                    df[str(data_year)] = df[str(data_year)] / max_value
                else:
                    # If max_value is NaN or zero, set max_value to 0 and fill the column with zeros
                    max_value = 0
                    df[str(data_year)].fillna(0, inplace=True)
                    print(f"Filled column with zeros for file: {csv_file_path}")
                
                # Save the maximum value in the 'Maximum_Level' column of df_reference
                df_reference.at[index, 'Maximum_Level'] = max_value
                
                # Save the updated CSV file
                df.to_csv(csv_file_path, index=False)
                
                # Print a message indicating that the file was processed
                print(f"Processed and normalized file: {csv_file_path}")
            else:
                print(f"Column {data_year} not found in {csv_file_path}")
                # Ensure the 'Maximum_Level' is set to 0 in df_reference if the column is not found
                df_reference.at[index, 'Maximum_Level'] = 0
        else:
            print(f"File not found: {csv_file_path}")
            # Ensure the 'Maximum_Level' is set to 0 in df_reference if the file is not found
            df_reference.at[index, 'Maximum_Level'] = 0
    
    # Save the updated reference data with the 'Maximum_Level' column
    df_reference.to_csv(reference_data_file_path, index=False)
    print(f"Updated reference data saved to: {reference_data_file_path}")

normalize_column(data_year, reference_data_file_path)

/tmp/ipykernel_69186/3430833264.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[str(data_year)].fillna(0, inplace=True)
/tmp/ipykernel_69186/3430833264.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

Processed and normalized file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/2023_1.csv
Filled column with zeros for file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/2023_1.csv
Processed and normalized file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/2023_1.csv
Processed and normalized file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/2023_1.csv
Processed and normalized file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/2023_1.csv
Filled column with zeros for file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/2023_1.csv
Processed and normalized file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/2023_1.csv
Filled column with zeros for file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CZ/2023_1.csv
Processed and normalized file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CZ/2023_1.csv
Filled column with zeros for file:

/tmp/ipykernel_69186/3430833264.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[str(data_year)].fillna(0, inplace=True)
/tmp/ipykernel_69186/3430833264.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Getting the corresponding time format.
<div/>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
As the full storage capacity data for all the countries are given by a time resolution of a week i.e. 53/54 weeks per year. So it is going to add the corresponding date to each week according to the analized year i.e. value specified in the data_year variable.
</div>

In [18]:
# Read the reference data CSV file
df_reference = pd.read_csv(reference_data_file_path)

# Iterate over each row in the DataFrame
for index, row in df_reference.iterrows():
    raw_data_file_path = row['Single_Raw_Data_File_Path']

    # Read the CSV file
    df = pd.read_csv(raw_data_file_path)

    # Extract week number from the 'Week' column
    df['Week'] = df['Week'].str.extract(r'(\d+)').astype(int)  # Extract digits from the string and convert to int

    # Calculate the dates
    start_date = pd.to_datetime(f'{data_year}-01-01')  # Start from January 1st of the specified year
    df['Dispa-SET_Date'] = start_date + pd.to_timedelta((df['Week'] - 1) * 7, unit='D')  # Add the corresponding number of weeks

    # Convert to string in the desired format
    df['Dispa-SET_Date'] = df['Dispa-SET_Date'].dt.strftime('%Y-%m-%d 00:00:00+00:00')

    # Save the modified DataFrame back to the CSV file
    df.to_csv(raw_data_file_path, index=False)

    print(f"Processed CSV file '{raw_data_file_path}'")

print("Processing completed for all CSV files.")

Processed CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/2023_1.csv'
Processed CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/2023_1.csv'
Processed CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/2023_1.csv'
Processed CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/2023_1.csv'
Processed CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/2023_1.csv'
Processed CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CZ/2023_1.csv'
Processed CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DE/2023_1.csv'
Processed CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DK/2023_1.csv'
Processed CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EE/2023_1.csv'
Processed CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EL/2023_1.csv'
Processed CSV file '/home/ray/Dispa-SET_

<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [19]:
print (f"dispaSET_unleash_folder_name:                              {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                              {dispaSET_unleash_folder_path}")
print (f"data_year:                                                 {data_year}")
print (f"reference_data_folder_path:                                {reference_data_folder_path}")
print (f"reference_data_file_name:                                  {reference_data_file_name}")
print (f"reference_data_file_path:                                  {reference_data_file_path}")
print (f"reservoir_level_folder_path:                               {reservoir_level_folder_path}")
print (f"reservoir_level_raw_data_folder_path:                      {reservoir_level_raw_data_folder_path}")
print (f"standard_time:                                             {standard_time}")

dispaSET_unleash_folder_name:                              Dispa-SET_Unleash
dispaSET_unleash_folder_path:                              /home/ray/Dispa-SET_Unleash
data_year:                                                 2023
reference_data_folder_path:                                /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/
reference_data_file_name:                                  Reference_Data.csv
reference_data_file_path:                                  /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Reference_Data.csv
reservoir_level_folder_path:                               /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/
reservoir_level_raw_data_folder_path:                      /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Series_Raw_Data_Source/
standard_time:                                             UTC


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
6.3. Time Step
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Since the raw data is given by week, it is needed interpolate the values throughout a whole year.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
For this purpouse the raw that from the availability factor folders is used.
</div>
<br>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Defining needed variables.
</div>

In [20]:
# Additional string to be appended
additional_path_3 = "/RawData/AvailabilityFactors/Time_Series_Raw_Data_Source/"

# Construct the Raw Data Source variable
raw_rata_source_folder_path = dispaSET_unleash_folder_path + additional_path_3

print("raw_rata_source_folder_path:", raw_rata_source_folder_path)

raw_rata_source_folder_path: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/Time_Series_Raw_Data_Source/


<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 16px; font-family: TimesNewRoman; color:skyblue">
6.3.1. Reference Raw Data for Time Step 
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Writting the 'Time_Series_Raw_Data_File_Path' column to the 'reference_data_file_path' file.
<br>
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color: skyblue">
This columns specify the path to the corresponding raw data csv file sources.
</div>

In [21]:
# Read the CSV file into a DataFrame
df = pd.read_csv(reference_data_file_path)

# Create the new column with the required file paths
df['Time_Series_Raw_Data_File_Path'] = df.apply(
    lambda row: f"{raw_rata_source_folder_path}{standard_time}/{row['Dispa-SET_Code']}/{data_year}_1.csv", axis=1)

# Save the updated DataFrame back to a CSV file
df.to_csv(reference_data_file_path, index=False)

print(f"Updated CSV file saved to {reference_data_file_path}")

Updated CSV file saved to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Reference_Data.csv


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Extracting the corresponding raw data time step reference file to be processed.
<br>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color: skyblue">
Updating the 'Time_Series_Raw_Data_File_Path' column of the 'reference_data_file_path' with the extracted raw data csv file sources paths.
</div>

In [22]:
def copy_and_organize_files(reference_data_file_path, reference_data_folder_path):
    """Copies and organizes CSV files based on information in a CSV file.

    Args:
        reference_data_file_path: Path to the CSV file containing file paths and codes.
        reference_data_folder_path: Path to the destination folder.
    """

    # Load the CSV file into a DataFrame
    df = pd.read_csv(reference_data_file_path)

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Get the source file path and Dispa-SET code
        source_file_path = row['Time_Series_Raw_Data_File_Path']
        dispa_set_code = row['Dispa-SET_Code']

        # Construct the destination folder path
        destination_folder = os.path.join(reference_data_folder_path, "Time_Step_Data_Reference", dispa_set_code)

        # Create the destination folder if it doesn't exist
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)

        # Construct the full destination file path
        destination_file_path = os.path.join(destination_folder, os.path.basename(source_file_path))

        # Copy the file from the source to the destination
        print(f"Copying file: {source_file_path} to {destination_file_path}")
        shutil.copy2(source_file_path, destination_file_path)

        # Update the DataFrame with the new file path
        df.at[index, 'Time_Series_Raw_Data_File_Path'] = destination_file_path

    # Save the updated DataFrame back to the CSV file
    df.to_csv(reference_data_file_path, index=False)

# Example usage
copy_and_organize_files(reference_data_file_path, reference_data_folder_path)

Copying file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/Time_Series_Raw_Data_Source/UTC/AT/2023_1.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/AT/2023_1.csv
Copying file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/Time_Series_Raw_Data_Source/UTC/BE/2023_1.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/BE/2023_1.csv
Copying file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/Time_Series_Raw_Data_Source/UTC/BG/2023_1.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/BG/2023_1.csv
Copying file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/Time_Series_Raw_Data_Source/UTC/CH/2023_1.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/CH/2023_1.csv
Copying file: /home/ray/Dispa-SET_Unleash/RawData/AvailabilityFactors/Time_Series_Raw_Data_Source/UTC/CY/2023_1.csv to /home

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Writing to the 'reference_data_file_path' the path to the country folders that content all the data realted to the time step reference file in order to use them for next processing stages.
</div>

In [23]:
def extract_zone_folder_paths(reference_data_file_path, data_year):
    """Extracts zone folder paths and creates a new column in the DataFrame.

    Args:
        reference_data_file_path (str): Path to the reference data CSV file.
        data_year (str): The year for the target CSV files.
    """

    df = pd.read_csv(reference_data_file_path)

    for index, row in df.iterrows():
        file_path = row['Time_Series_Raw_Data_File_Path']
        zone_folder_path = os.path.dirname(file_path)
        df.at[index, 'Time_Step_Reference_Zone_Folder_Path'] = zone_folder_path
        print(f"Extracted zone folder path for row {index}: {zone_folder_path}")

    df.to_csv(reference_data_file_path, index=False)

extract_zone_folder_paths(reference_data_file_path, data_year)

Extracted zone folder path for row 0: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/AT
Extracted zone folder path for row 1: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/BE
Extracted zone folder path for row 2: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/BG
Extracted zone folder path for row 3: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/CH
Extracted zone folder path for row 4: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/CY
Extracted zone folder path for row 5: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/CZ
Extracted zone folder path for row 6: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/DE
Extracted zone folder path for row 7: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Da

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 16px; font-family: TimesNewRoman; color:skyblue">
    6.3.2. Time Step Reference Data Processing
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Adding column names.
<br>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color: skyblue">
All the time step reference data were downloaded without columns identification, the corresponding header to each column is added.
</div>

In [24]:
# Define the headers
headers = [
    "MTU", "Biomass_Actual_Aggregated", "Biomass_Actual_Consumption",
    "Fossil_Brown_coal-Lignite_Actual_Aggregated", "Fossil_Brown_coal-Lignite_Actual_Consumption",
    "Fossil_Coal-derived_gas_Actual_Aggregated", "Fossil_Coal-derived_gas_Actual_Consumption",
    "Fossil_Gas_Actual_Aggregated", "Fossil_Gas_Actual_Consumption",
    "Fossil_Hard_coal_Actual_Aggregated", "Fossil_Hard_coal_Actual_Consumption",
    "Fossil_Oil_Actual_Aggregated", "Fossil_Oil_Actual_Consumption",
    "Fossil_Oil_shale_Actual_Aggregated", "Fossil_Oil_shale_Actual_Consumption",
    "Fossil_Peat_Actual_Aggregated", "Fossil_Peat_Actual_Consumption",
    "Geothermal_Actual_Aggregated", "Geothermal_Actual_Consumption",
    "Hydro_Pumped_Storage_Actual_Aggregated", "Hydro_Pumped_Storage_Actual_Consumption",
    "Hydro_Run-of-river_and_poundage_Actual_Aggregated", "Hydro_Run-of-river_and_poundage_Actual_Consumption",
    "Hydro_Water_Reservoir_Actual_Aggregated", "Hydro_Water_Reservoir_Actual_Consumption",
    "Marine_Actual_Aggregated", "Marine_Actual_Consumption",
    "Nuclear_Actual_Aggregated", "Nuclear_Actual_Consumption",
    "Other_Actual_Aggregated", "Other_Actual_Consumption",
    "Other_renewable_Actual_Aggregated", "Other_renewable_Actual_Consumption",
    "Solar_Actual_Aggregated", "Solar_Actual_Consumption",
    "Waste_Actual_Aggregated", "Waste_Actual_Consumption",
    "Wind_Offshore_Actual_Aggregated", "Wind_Offshore_Actual_Consumption",
    "Wind_Onshore_Actual_Aggregated", "Wind_Onshore_Actual_Consumption"
]

# Read the CSV file containing the paths
df = pd.read_csv(reference_data_file_path)

success = True

# Iterate through each file path in the column 'Time_Series_Raw_Data_File_Path'
for index, row in df.iterrows():
    file_path = row['Time_Series_Raw_Data_File_Path']
    
    # Check if the file exists
    if os.path.exists(file_path):
        # Open the existing CSV file in read mode
        with open(file_path, 'r', newline='') as file:
            # Read the existing content
            reader = csv.reader(file)
            rows = list(reader)

        # Insert a new empty row at the beginning
        rows.insert(0, [])

        # Write the updated content back to the CSV file
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(rows)
        
        print(f"Empty row added at the beginning of file '{file_path}'.")

        # Read the file content again after adding the empty row
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Find the index of the first empty row
        empty_row_index = next((i for i, line in enumerate(lines) if line.strip() == ""), None)

        # If an empty row is found, copy the headers to it
        if empty_row_index is not None:
            lines[empty_row_index] = ','.join(headers) + '\n'

            # Write the updated lines back to the file
            with open(file_path, 'w') as file:
                file.writelines(lines)
        else:
            print(f"No empty row found in file '{file_path}'. Headers not copied.")
    else:
        success = False
        print(f"File not found: {file_path}")

if success:
    print("Empty rows added and headers copied successfully to the first empty row of all files.")
else:
    print("Some errors occurred while processing the files.")

Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/AT/2023_1.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/BE/2023_1.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/BG/2023_1.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/CH/2023_1.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/CY/2023_1.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/CZ/2023_1.csv'.
Empty row added at the beginning of file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/DE/2023_

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color: skyblue">
6.3.3. Time Step Reference Data Time Resolution
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
Identifying the time spept of all the reference data files.
<br>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color: skyblue">
Additional columns that indicates the Year, Mounth, Day, Hour and minute of the data are added to the files.
</div>

In [25]:
# Read the CSV file containing the paths
df = pd.read_csv(reference_data_file_path)

# Function to validate and convert MTU parts to integers
def safe_int_conversion(value):
    try:
        return int(value)
    except ValueError:
        return None  # or some default value like 0

# Iterate through each file path in the 'Time_Series_Raw_Data_File_Path' column
for file_path in df['Time_Series_Raw_Data_File_Path']:
    # Read the CSV file
    file_df = pd.read_csv(file_path)

    # Add new columns 'Year', 'Month', 'Day', 'Hour', and 'Minute'
    file_df['Year'] = data_year
    file_df['Month'] = ''
    file_df['Day'] = ''

    # Ensure the 'MTU' column is treated as a string and is 5 characters long
    if file_df['MTU'].dtype != 'object':
        file_df['MTU'] = file_df['MTU'].astype(str)

    # Ensure all strings in 'MTU' column are at least 5 characters long
    file_df['MTU'] = file_df['MTU'].str.zfill(5)

    # Extract hour and minute from 'MTU' column with safe conversion
    file_df['Hour'] = file_df['MTU'].str[:2].apply(safe_int_conversion)
    file_df['Minute'] = file_df['MTU'].str[3:5].apply(safe_int_conversion)

    # Reorder the columns to have 'Year', 'Month', 'Day', 'Hour', and 'Minute' first
    file_df = file_df[['Year', 'Month', 'Day', 'Hour', 'Minute'] + [col for col in file_df.columns if col not in ['Year', 'Month', 'Day', 'Hour', 'Minute']]]

    # Write the updated DataFrame back to the CSV file
    file_df.to_csv(file_path, index=False)

    print(f"Updated file: {file_path}")

print("All files updated successfully.")


Updated file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/AT/2023_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/BE/2023_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/BG/2023_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/CH/2023_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/CY/2023_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/CZ/2023_1.csv


/tmp/ipykernel_69186/1429228844.py:14: DtypeWarning: Columns (27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file_path)


Updated file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/DE/2023_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/DK/2023_1.csv


/tmp/ipykernel_69186/1429228844.py:14: DtypeWarning: Columns (27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file_path)


Updated file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/EE/2023_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/EL/2023_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/ES/2023_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/FI/2023_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/FR/2023_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/HR/2023_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/HU/2023_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/IE/2023_1.csv
Updated file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Dat

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Extracting the time stept data.
<br>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
The rows corresponding to a time resolution of 1 hour, 30 minutes and 15 minutes are extracted and added to new files with the suffix _1h, _30min, and _15min.
</div>

In [26]:
def process_csv_file(csv_file_path):
    print(f"Processing file: {csv_file_path}")
    
    # Load the CSV file
    df = pd.read_csv(csv_file_path)
    
    # Count the number of rows
    num_rows = len(df)
    
    # Define file name without extension
    file_name_no_ext = os.path.splitext(os.path.basename(csv_file_path))[0]
    
    # Get the directory of the current CSV file
    base_dir = os.path.dirname(csv_file_path)
    
    # Check conditions and process accordingly
    if num_rows in [35040, 35136, 34544]:
        print(f"File has 35040 or 34544 rows. Processing...")
        process_35040_34544(df, file_name_no_ext, base_dir)
    elif num_rows in [17520, 17522, 17568, 17570]:
        print(f"File has 17520 or 17522 rows. Processing...")
        process_17520_17522(df, file_name_no_ext, base_dir)
    elif num_rows in [8760, 8761, 8784, 8785]:
        print(f"File has 8760 or 8761 rows. Processing...")
        process_8760_8761(df, file_name_no_ext, base_dir)
    else:
        print(f"File has {num_rows} rows. No specific conditions matched. Processing as default...")
        process_default(df, file_name_no_ext, base_dir)

def process_35040_34544(df, file_name_no_ext, base_dir):
    # Create new file paths
    suffixes = ['_1h', '_30min', '_15min']
    new_file_paths = [os.path.join(base_dir, file_name_no_ext + suffix + '.csv') for suffix in suffixes]
    
    # Write data to new files
    for new_file_path in new_file_paths:
        df.to_csv(new_file_path, index=False)
    
    # Filter rows and write to corresponding files
    df_minute_zero = df[df['Minute'] == 0]
    df_minute_zero.to_csv(os.path.join(base_dir, file_name_no_ext + '_1h.csv'), index=False)
    
    df_minute_zero_or_thirty = df[df['Minute'].isin([0, 30])]
    df_minute_zero_or_thirty.to_csv(os.path.join(base_dir, file_name_no_ext + '_30min.csv'), index=False)
    
    print("Processed successfully.")

def process_17520_17522(df, file_name_no_ext, base_dir):
    # Create new file paths
    suffixes = ['_1h', '_30min']
    new_file_paths = [os.path.join(base_dir, file_name_no_ext + suffix + '.csv') for suffix in suffixes]
    
    # Write data to new files
    for new_file_path in new_file_paths:
        df.to_csv(new_file_path, index=False)
    
    # Filter rows and write to corresponding files
    df_minute_zero = df[df['Minute'] == 0]
    df_minute_zero.to_csv(os.path.join(base_dir, file_name_no_ext + '_1h.csv'), index=False)
    
    print("Processed successfully.")

def process_8760_8761(df, file_name_no_ext, base_dir):
    # Create new file path
    new_file_path = os.path.join(base_dir, file_name_no_ext + '_1h.csv')
    
    # Write data to new file
    df.to_csv(new_file_path, index=False)
    
    print("Processed successfully.")

def process_default(df, file_name_no_ext, base_dir):
    # Create new file path
    new_file_path = os.path.join(base_dir, file_name_no_ext + '_1h.csv')
    
    # Filter rows and write to corresponding files
    df_minute_zero = df[df['Minute'] == 0]
    df_minute_zero.to_csv(os.path.join(base_dir, file_name_no_ext + '_1h.csv'), index=False)
    
    print("Processed successfully.")

# File path
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"

# Load the CSV file
standard_time_data_df = pd.read_csv(reference_data_file_path)

# Iterate over each CSV file specified in 'Time_Series_Raw_Data_File_Path'
for csv_file_path in standard_time_data_df['Time_Series_Raw_Data_File_Path']:
    process_csv_file(csv_file_path)

Processing file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/AT/2023_1.csv
File has 35040 or 34544 rows. Processing...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/BE/2023_1.csv
File has 8760 or 8761 rows. Processing...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/BG/2023_1.csv
File has 8760 or 8761 rows. Processing...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/CH/2023_1.csv
File has 8760 or 8761 rows. Processing...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/CY/2023_1.csv
File has 17520 or 17522 rows. Processing...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Tim

/tmp/ipykernel_69186/953703549.py:5: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)


Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/DK/2023_1.csv
File has 8760 or 8761 rows. Processing...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/EE/2023_1.csv
File has 35040 or 34544 rows. Processing...


/tmp/ipykernel_69186/953703549.py:5: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)


Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/EL/2023_1.csv
File has 8760 or 8761 rows. Processing...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/ES/2023_1.csv
File has 35040 or 34544 rows. Processing...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/FI/2023_1.csv
File has 24960 rows. No specific conditions matched. Processing as default...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/FR/2023_1.csv
File has 8760 or 8761 rows. Processing...
Processed successfully.
Processing file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/HR/2023_1.csv
File has 11064 rows. No specific conditions matched. Processing as default...
Processed s

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Filling the correspondig time stept.
<br>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
For the next stepts of the formating process it is necesary identify the path of the files where all the data of the 30 Dispa-SET countries are going to be storaged differencing them into time stepts (1 hour, 30 min, and 15 min)
</div>

In [27]:
# Define file paths and variables
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"
#data_year = 2023

# Read the standard time data CSV file
df = pd.read_csv(reference_data_file_path)

# Define function to search for files in directory
def search_files(directory, prefix):
    for file in os.listdir(directory):
        if file.startswith(prefix):
            return os.path.join(directory, file)
    return None

# Add new columns
df[f"{data_year}_1h_File_Path"] = ""
df[f"{data_year}_30min_File_Path"] = ""
df[f"{data_year}_15min_File_Path"] = ""

# Iterate over Zone_Folder_Path column
for index, row in df.iterrows():
    zone_folder_path = row['Time_Step_Reference_Zone_Folder_Path']
    if os.path.exists(zone_folder_path):
        # Search for files in the directory
        hour_file_path = search_files(zone_folder_path, f"{data_year}_1_1h.csv")
        if hour_file_path:
            df.at[index, f"{data_year}_1h_File_Path"] = hour_file_path
            
        min30_file_path = search_files(zone_folder_path, f"{data_year}_1_30min.csv")
        if min30_file_path:
            df.at[index, f"{data_year}_30min_File_Path"] = min30_file_path
            
        min15_file_path = search_files(zone_folder_path, f"{data_year}_1_15min.csv")
        if min15_file_path:
            df.at[index, f"{data_year}_15min_File_Path"] = min15_file_path

# Save the updated DataFrame back to the same CSV file
df.to_csv(reference_data_file_path, index=False)

print("CSV file updated successfully.")

CSV file updated successfully.


<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Fulling the corresponding time step to each file.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Not all the countries have data in a resolution of 15 mimuntes, 30 minutes or 1 hour, so, all the next three cells all the files of the corresponding time stept are going to be fullfilling in their corresponding columns of Minute, Hour, Day and Month of the year specified in the variable data_year.
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
<br>
1 Hour time stept.
</div>

In [28]:
# Specify the year
year = data_year

# Load the CSV file containing paths
df_paths = pd.read_csv(reference_data_file_path)

# Iterate over each row in the DataFrame
for index, row in df_paths.iterrows():
    # Get the path from the corresponding column
    file_path_column_name = f"{year}_1h_File_Path"  # Dynamically construct the column name based on the year
    file_path = row[file_path_column_name]
    
    # Check if the path exists and is not NaN
    if isinstance(file_path, str) and os.path.exists(file_path):
        # Load the existing CSV file
        df = pd.read_csv(file_path)

        # Generate a date range for the entire year
        dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')

        # Extract month, day, and hour from the date range
        months = [date.month for date in dates]
        days = [date.day for date in dates]
        hours = [date.hour % 24 for date in dates]  # Cycle through hours (0-23)

        # Update the DataFrame with the generated data
        df['Month'] = months
        df['Day'] = days
        df['Hour'] = hours

        # Save the updated DataFrame back to the same CSV file, overwriting the original file
        df.to_csv(file_path, index=False)

        print(f"CSV file updated successfully: {file_path}")
    else:
        print(f"No valid path specified in row {index + 1}. Skipping...")

CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/AT/2023_1_1h.csv
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/BE/2023_1_1h.csv


/tmp/ipykernel_69186/3348126855.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')
/tmp/ipykernel_69186/3348126855.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')
/tmp/ipykernel_69186/3348126855.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')


CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/BG/2023_1_1h.csv
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/CH/2023_1_1h.csv
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/CY/2023_1_1h.csv


/tmp/ipykernel_69186/3348126855.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')
/tmp/ipykernel_69186/3348126855.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')
/tmp/ipykernel_69186/3348126855.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')


CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/CZ/2023_1_1h.csv
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/DE/2023_1_1h.csv


...

CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/SE/2023_1_1h.csv


/tmp/ipykernel_69186/3348126855.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')


CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/SI/2023_1_1h.csv
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/SK/2023_1_1h.csv


/tmp/ipykernel_69186/3348126855.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')
/tmp/ipykernel_69186/3348126855.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:00:00', freq='H')


CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/UK/2023_1_1h.csv


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
30 Minutes time stept.
</div>

In [29]:
# Specify the year
year = data_year

# Load the CSV file containing paths
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"
df_paths = pd.read_csv(reference_data_file_path)

# Iterate over each row in the DataFrame
for index, row in df_paths.iterrows():
    # Dynamically construct the column name based on the year
    file_path_column_name = f"{year}_30min_File_Path"
    
    # Get the path from the corresponding column
    file_path = row[file_path_column_name]
    
    # Check if the path exists and is not NaN
    if isinstance(file_path, str) and os.path.exists(file_path):
        # Load the existing CSV file
        df = pd.read_csv(file_path)

        # Generate a date range for the entire year with a time step of 30 minutes
        dates_30min = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:59:59', freq='30T')

        # Extract month, day, and hour from the date range with 30-minute time step
        months_30min = [date.month for date in dates_30min]
        days_30min = [date.day for date in dates_30min]
        hours_30min = [date.hour % 24 for date in dates_30min]  # Cycle through hours (0-23)

        # Update the DataFrame with the generated data for 30-minute time step
        df['Month'] = months_30min
        df['Day'] = days_30min
        df['Hour'] = hours_30min

        # Save the updated DataFrame back to the same CSV file, overwriting the original file
        df.to_csv(file_path, index=False)

        print(f"CSV file updated successfully with 30-minute time step: {file_path}")
    else:
        print(f"No valid path specified in row {index + 1}. Skipping...")

/tmp/ipykernel_69186/2732984526.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates_30min = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:59:59', freq='30T')


CSV file updated successfully with 30-minute time step: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/AT/2023_1_30min.csv
No valid path specified in row 2. Skipping...
No valid path specified in row 3. Skipping...
No valid path specified in row 4. Skipping...


/tmp/ipykernel_69186/2732984526.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates_30min = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:59:59', freq='30T')
/tmp/ipykernel_69186/2732984526.py:19: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


CSV file updated successfully with 30-minute time step: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/CY/2023_1_30min.csv
No valid path specified in row 6. Skipping...
CSV file updated successfully with 30-minute time step: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/DE/2023_1_30min.csv
No valid path specified in row 8. Skipping...


/tmp/ipykernel_69186/2732984526.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates_30min = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:59:59', freq='30T')
/tmp/ipykernel_69186/2732984526.py:19: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_69186/2732984526.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates_30min = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:59:59', freq='30T')


...

CSV file updated successfully with 30-minute time step: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/LU/2023_1_30min.csv
No valid path specified in row 20. Skipping...
No valid path specified in row 21. Skipping...
CSV file updated successfully with 30-minute time step: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/NL/2023_1_30min.csv
No valid path specified in row 23. Skipping...
No valid path specified in row 24. Skipping...
No valid path specified in row 25. Skipping...


/tmp/ipykernel_69186/2732984526.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates_30min = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:59:59', freq='30T')


CSV file updated successfully with 30-minute time step: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/RO/2023_1_30min.csv
No valid path specified in row 27. Skipping...
No valid path specified in row 28. Skipping...
No valid path specified in row 29. Skipping...


/tmp/ipykernel_69186/2732984526.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates_30min = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:59:59', freq='30T')


CSV file updated successfully with 30-minute time step: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/UK/2023_1_30min.csv


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
15 Minutes time stept.
</div>

In [30]:
# Specify the year
year = data_year

# Load the CSV file containing paths
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"
df_paths = pd.read_csv(reference_data_file_path)

# Iterate over each row in the DataFrame
for index, row in df_paths.iterrows():
    # Dynamically construct the column name based on the year
    file_path_column_name = f"{year}_15min_File_Path"
    
    # Get the path from the corresponding column
    file_path = row[file_path_column_name]
    
    # Check if the path exists and is not NaN
    if isinstance(file_path, str) and os.path.exists(file_path):
        # Load the existing CSV file
        df = pd.read_csv(file_path)

        # Generate a date range for the entire year with a time step of 15 minutes
        dates_15min = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:59:59', freq='15T')

        # Extract month, day, and hour from the date range with 15-minute time step
        months_15min = [date.month for date in dates_15min]
        days_15min = [date.day for date in dates_15min]
        hours_15min = [date.hour % 24 for date in dates_15min]  # Cycle through hours (0-23)

        # Update the DataFrame with the generated data for 15-minute time step
        df['Month'] = months_15min
        df['Day'] = days_15min
        df['Hour'] = hours_15min

        # Save the updated DataFrame back to the same CSV file, overwriting the original file
        df.to_csv(file_path, index=False)

        print(f"CSV file updated successfully with 15-minute time step: {file_path}")
    else:
        print(f"No valid path specified in row {index + 1}. Skipping...")

/tmp/ipykernel_69186/1337634457.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates_15min = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:59:59', freq='15T')


CSV file updated successfully with 15-minute time step: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/AT/2023_1_15min.csv
No valid path specified in row 2. Skipping...
No valid path specified in row 3. Skipping...
No valid path specified in row 4. Skipping...
No valid path specified in row 5. Skipping...
No valid path specified in row 6. Skipping...


/tmp/ipykernel_69186/1337634457.py:19: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_69186/1337634457.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates_15min = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:59:59', freq='15T')


CSV file updated successfully with 15-minute time step: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/DE/2023_1_15min.csv
No valid path specified in row 8. Skipping...


/tmp/ipykernel_69186/1337634457.py:19: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_69186/1337634457.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates_15min = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:59:59', freq='15T')


...

CSV file updated successfully with 15-minute time step: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/LU/2023_1_15min.csv
No valid path specified in row 20. Skipping...
No valid path specified in row 21. Skipping...


/tmp/ipykernel_69186/1337634457.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates_15min = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:59:59', freq='15T')


CSV file updated successfully with 15-minute time step: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/NL/2023_1_15min.csv
No valid path specified in row 23. Skipping...
No valid path specified in row 24. Skipping...
No valid path specified in row 25. Skipping...


/tmp/ipykernel_69186/1337634457.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates_15min = pd.date_range(start=f'{year}-01-01 00:00:00', end=f'{year}-12-31 23:59:59', freq='15T')


CSV file updated successfully with 15-minute time step: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/RO/2023_1_15min.csv
No valid path specified in row 27. Skipping...
No valid path specified in row 28. Skipping...
No valid path specified in row 29. Skipping...
No valid path specified in row 30. Skipping...


<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Adding the Dispa-SET Time Stept format.
</div>
<div style="text-align: justify; margin-left: 1.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
The format that Dispa-SET reads the time stept is the following, 0000-00-00 00:00:00+00:00 where the first part represents the date and the second part represents the time, so the next script add to each row of the correspongind file (1 hour file, 30 minutes file and 15 minutes file for each country) the time stept acording this format.
</div>

In [31]:
# Define a function to create Dispa_SET_Time_Step column
def create_Dispa_SET_Time_Step(row):
    # Ensure values are integers (or default to 0 if NaN) before formatting
    year = int(row['Year']) if pd.notna(row['Year']) else 0
    month = int(row['Month']) if pd.notna(row['Month']) else 0
    day = int(row['Day']) if pd.notna(row['Day']) else 0
    hour = int(row['Hour']) if pd.notna(row['Hour']) else 0
    minute = int(row['Minute']) if pd.notna(row['Minute']) else 0
    
    # Ensure two-digit format for Day, Month, Hour, and Minute
    date_part = f"{year:04d}-{month:02d}-{day:02d}"
    time_part = f"{hour:02d}:{minute:02d}:00+00:00"
    return date_part + " " + time_part

# Specify the year
#data_year = 2023

# Load the CSV file containing file paths
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"
df_paths = pd.read_csv(reference_data_file_path)

# Iterate over each row in the DataFrame
for index, row in df_paths.iterrows():
    # Get the file paths from the corresponding columns
    hour_file_path = row[f"{data_year}_1h_File_Path"]
    min_30_file_path = row[f"{data_year}_30min_File_Path"]
    min_15_file_path = row[f"{data_year}_15min_File_Path"]
    
    # Process each file path if it exists
    for file_path in [hour_file_path, min_30_file_path, min_15_file_path]:
        # Check if the path exists and is not NaN
        if isinstance(file_path, str) and os.path.exists(file_path):
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)
            
            # Apply the function to create the Dispa_SET_Time_Step column
            df['Dispa_SET_Time_Step'] = df.apply(create_Dispa_SET_Time_Step, axis=1)
            
            # Save the modified DataFrame back to the CSV file
            df.to_csv(file_path, index=False)
            
            print(f"CSV file updated successfully: {file_path}")
        else:
            print(f"No valid path specified in row {index + 1}. Skipping...")

CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/AT/2023_1_1h.csv
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/AT/2023_1_30min.csv
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/AT/2023_1_15min.csv
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/BE/2023_1_1h.csv
No valid path specified in row 2. Skipping...
No valid path specified in row 2. Skipping...
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/BG/2023_1_1h.csv
No valid path specified in row 3. Skipping...
No valid path specified in row 3. Skipping...
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/CH/2023_1_1h.csv
No val

/tmp/ipykernel_69186/2714078988.py:34: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/DE/2023_1_30min.csv


/tmp/ipykernel_69186/2714078988.py:34: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/DE/2023_1_15min.csv
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/DK/2023_1_1h.csv
No valid path specified in row 8. Skipping...
No valid path specified in row 8. Skipping...
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/EE/2023_1_1h.csv


/tmp/ipykernel_69186/2714078988.py:34: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/EE/2023_1_30min.csv


/tmp/ipykernel_69186/2714078988.py:34: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/EE/2023_1_15min.csv
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/EL/2023_1_1h.csv
No valid path specified in row 10. Skipping...
No valid path specified in row 10. Skipping...
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/ES/2023_1_1h.csv
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/ES/2023_1_30min.csv
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/ES/2023_1_15min.csv
CSV file updated successfully: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Time_Step_Data_Reference/FI/2023_1_1h.csv
No valid path specified in row 12. Skipping...
No valid path specified in row 12. Skipping...

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Erasing the unnecesary columns and ordering.
</div>

In [32]:
# Define the function to process each CSV file
def process_csv_file(row, time_step_column):
    csv_file = row[time_step_column]
    if pd.notnull(csv_file) and os.path.exists(csv_file):
        # Read the CSV file into a DataFrame
        df_csv = pd.read_csv(csv_file)
        
        # Drop specified columns: Year, Month, Day, Hour, Minute, MTU
        columns_to_drop = ['Year', 'Month', 'Day', 'Hour', 'Minute', 'MTU']
        df_csv = df_csv.drop(columns=columns_to_drop, errors='ignore')
        
        # Move the column 'Dispa_SET_Time_Step' to the first position
        if 'Dispa_SET_Time_Step' in df_csv.columns:
            columns = list(df_csv.columns)
            columns.remove('Dispa_SET_Time_Step')
            df_csv = df_csv[['Dispa_SET_Time_Step'] + columns]
        
        # Save the changes back to the CSV file
        df_csv.to_csv(csv_file, index=False)
        
# Define the path to the CSV file containing the file paths
#standard_time_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv"

# Read the CSV file containing the paths
df = pd.read_csv(reference_data_file_path)

# Convert data_year to a string
data_year = str(data_year)

# Define the column names for the file paths
hour_file_column = data_year + '_1h_File_Path'
thirty_min_file_column = data_year + '_30min_File_Path'
fifteen_min_file_column = data_year + '_15min_File_Path'

# Process the one-hour file
df.apply(lambda row: process_csv_file(row, hour_file_column), axis=1)

# Process the thirty-minute file
df.apply(lambda row: process_csv_file(row, thirty_min_file_column), axis=1)

# Process the fifteen-minute file
df.apply(lambda row: process_csv_file(row, fifteen_min_file_column), axis=1)

/tmp/ipykernel_69186/2678497721.py:6: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv = pd.read_csv(csv_file)
/tmp/ipykernel_69186/2678497721.py:6: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv = pd.read_csv(csv_file)
/tmp/ipykernel_69186/2678497721.py:6: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv = pd.read_csv(csv_file)
/tmp/ipykernel_69186/2678497721.py:6: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv = pd.read_csv(csv_file)


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
dtype: object

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
7. Reservoirs Level Files
</div>
<br>
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 16px; font-family: TimesNewRoman; color:skyblue">
7.1. Reservoirs Level Clean Files Creation
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Creating all the clean csv files where the final data of the reservoir levels have to be gathered.
<br>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
The next three cells are use to create the content folders (1h, 30 min and/or 15min) and the cleaned csv files according the time step provided by the web source (ENTSO E).
</div>

In [33]:
def process_csv_files(reference_data_file_path, data_year):
    """Copies, renames, and processes CSV files.

    Args:
        reference_data_file_path (str): Path to the reference data CSV file.
        data_year (str): The year for the target CSV files.
    """

    df = pd.read_csv(reference_data_file_path)

    for index, row in df.iterrows():
        time_step_folder = row['Time_Step_Reference_Zone_Folder_Path']
        zone_folder = row['Zone_Folder_Path']

        for time_interval in ['1h', '15min', '30min']:
            source_file = os.path.join(time_step_folder, f"{data_year}_1_{time_interval}.csv")
            destination_folder = os.path.join(zone_folder, time_interval)
            old_destination_file = os.path.join(destination_folder, f"{data_year}_1_{time_interval}.csv")
            new_destination_file = os.path.join(destination_folder, f"{data_year}.csv")

            if not os.path.exists(destination_folder):
                os.makedirs(destination_folder)

            if os.path.exists(source_file):
                data = pd.read_csv(source_file)
                first_column = data.iloc[:, 0]
                first_column = first_column.to_frame(name='Value')
                first_column['HPHS'] = 0
                first_column['HDAM'] = 0

                if os.path.exists(old_destination_file):
                    os.rename(old_destination_file, new_destination_file)
                    print(f"Renamed file: {old_destination_file} to {new_destination_file}")

                first_column.to_csv(new_destination_file, index=False, mode='a', header=not os.path.exists(new_destination_file))
                print(f"Copied and processed data to {new_destination_file}")

process_csv_files(reference_data_file_path, data_year)

Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/1h/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/15min/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/30min/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/1h/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/1h/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/1h/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/1h/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/30min/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CZ/1h/2023.csv
Copied and processed data to /home/ray/Dispa-S

/tmp/ipykernel_69186/2296304554.py:25: DtypeWarning: Columns (27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(source_file)


Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DE/30min/2023.csv


/tmp/ipykernel_69186/2296304554.py:25: DtypeWarning: Columns (27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(source_file)
/tmp/ipykernel_69186/2296304554.py:25: DtypeWarning: Columns (27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(source_file)


Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DK/1h/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EE/1h/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EE/15min/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EE/30min/2023.csv


/tmp/ipykernel_69186/2296304554.py:25: DtypeWarning: Columns (27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(source_file)


Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EL/1h/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/ES/1h/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/ES/15min/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/ES/30min/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/FI/1h/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/FR/1h/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/HR/1h/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/HU/1h/2023.csv
Copied and processed data to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/HU/15min/2023.csv
Copied and processed data to /home/ray/Dispa-S

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Updating the reference_data file with the path to the clean files according the time step
</div>

In [34]:
def copy_first_column_and_add_columns(reference_data_file_path, data_year):
    """Copies the first column of CSV files to subfolders and adds new columns.

    Args:
        reference_data_file_path (str): Path to the reference data CSV file.
        data_year (str): The year for the target CSV files.
    """

    df = pd.read_csv(reference_data_file_path)

    for index, row in df.iterrows():
        time_step_folder = row['Time_Step_Reference_Zone_Folder_Path']
        zone_folder = row['Zone_Folder_Path']

        for time_interval in ['1h', '15min', '30min']:
            source_file = os.path.join(time_step_folder, f"{data_year}_1_{time_interval}.csv")
            destination_folder = os.path.join(zone_folder, time_interval)
            destination_file = os.path.join(destination_folder, f"{data_year}_1_{time_interval}.csv")

            if not os.path.exists(destination_folder):
                os.makedirs(destination_folder)

            if os.path.exists(source_file):
                data = pd.read_csv(source_file)
                first_column = data.iloc[:, 0]
                first_column = first_column.to_frame(name='Value')
                first_column['HPHS'] = 0  # Add HPHS column with initial values
                first_column['HDAM'] = 0  # Add HDAM column with initial values
                first_column.to_csv(destination_file, index=False)
                print(f"Copied first column and added HPHS, HDAM to {destination_file}")


copy_first_column_and_add_columns(reference_data_file_path, data_year)

Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/1h/2023_1_1h.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/15min/2023_1_15min.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/30min/2023_1_30min.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/1h/2023_1_1h.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/1h/2023_1_1h.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/1h/2023_1_1h.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/1h/2023_1_1h.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/30min

/tmp/ipykernel_69186/3298507601.py:24: DtypeWarning: Columns (27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(source_file)


Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DE/30min/2023_1_30min.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DK/1h/2023_1_1h.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EE/1h/2023_1_1h.csv


/tmp/ipykernel_69186/3298507601.py:24: DtypeWarning: Columns (27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(source_file)
/tmp/ipykernel_69186/3298507601.py:24: DtypeWarning: Columns (27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(source_file)


Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EE/15min/2023_1_15min.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EE/30min/2023_1_30min.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EL/1h/2023_1_1h.csv


/tmp/ipykernel_69186/3298507601.py:24: DtypeWarning: Columns (27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(source_file)


Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/ES/1h/2023_1_1h.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/ES/15min/2023_1_15min.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/ES/30min/2023_1_30min.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/FI/1h/2023_1_1h.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/FR/1h/2023_1_1h.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/HR/1h/2023_1_1h.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/HU/1h/2023_1_1h.csv
Copied first column and added HPHS, HDAM to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/HU/15min

In [35]:
# Read the reference data CSV file
df = pd.read_csv(reference_data_file_path)

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    zone_folder_path = row['Zone_Folder_Path']
    
    # Check if the zone folder path exists
    if os.path.exists(zone_folder_path):
        # Iterate over the required subfolders
        for subfolder in ['1h', '30min', '15min']:
            subfolder_path = os.path.join(zone_folder_path, subfolder)
            
            # Check if the subfolder exists
            if os.path.exists(subfolder_path):
                # Check if the CSV file for the data year exists in the subfolder
                csv_file_path = os.path.join(subfolder_path, f"{data_year}.csv")
                if os.path.exists(csv_file_path):
                    # Write the CSV file path to the corresponding column in the DataFrame
                    df.at[index, subfolder] = csv_file_path
                    print(f"CSV file '{data_year}.csv' found in '{subfolder_path}'.")
                else:
                    print(f"CSV file '{data_year}.csv' not found in '{subfolder_path}'.")
            else:
                print(f"Subfolder '{subfolder}' does not exist in '{zone_folder_path}'.")
    else:
        print(f"Zone folder path '{zone_folder_path}' does not exist.")

# Save the updated DataFrame to the reference data CSV file
df.to_csv(reference_data_file_path, index=False)
print("CSV file updated successfully.")

CSV file '2023.csv' found in '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/1h'.
CSV file '2023.csv' found in '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/30min'.
CSV file '2023.csv' found in '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/15min'.
CSV file '2023.csv' found in '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/1h'.
CSV file '2023.csv' not found in '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/30min'.
CSV file '2023.csv' not found in '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/15min'.
CSV file '2023.csv' found in '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/1h'.
CSV file '2023.csv' not found in '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/30min'.
CSV file '2023.csv' not found in '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/15min'.
CSV file '2023.csv' found in '/home/ray/Dispa-SET_Unleash/RawData/HydroData

/tmp/ipykernel_69186/1579858687.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/1h/2023.csv' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, subfolder] = csv_file_path
/tmp/ipykernel_69186/1579858687.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/30min/2023.csv' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, subfolder] = csv_file_path
/tmp/ipykernel_69186/1579858687.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/15min/2023.cs

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Eransing all the second and third column content excepting the headers and changin the first column header by 'Dispa-SET_Date
</div>

In [36]:
def modify_csv_files(reference_data_file_path):
    """Modifies CSV files by erasing values and renaming the first column.

    Args:
        reference_data_file_path (str): Path to the reference data CSV file.
    """

    df = pd.read_csv(reference_data_file_path)

    for index, row in df.iterrows():
        for column in ['1h', '30min', '15min']:
            csv_file_path = row[column]

            if pd.notna(csv_file_path):
                df_csv = pd.read_csv(csv_file_path)
                df_csv.iloc[:, 1:3] = ''
                df_csv.columns = ['Dispa-SET_Date'] + list(df_csv.columns[1:])
                df_csv.to_csv(csv_file_path, index=False)
                print(f"Values erased and first column renamed in CSV file '{csv_file_path}'.")

    print("Process completed successfully.")

modify_csv_files(reference_data_file_path)

Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/1h/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/30min/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/15min/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/1h/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/1h/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/1h/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/1h/2023.csv'.


/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_csv.iloc[:, 1:3] = ''
/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_csv.iloc[:, 1:3] = ''
/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_csv.iloc[:, 1:3] = ''
/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible

Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/30min/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CZ/1h/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DE/1h/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DE/30min/2023.csv'.


/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_csv.iloc[:, 1:3] = ''
/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_csv.iloc[:, 1:3] = ''
/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_csv.iloc[:, 1:3] = ''
/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible

Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DE/15min/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DK/1h/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EE/1h/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EE/30min/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EE/15min/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EL/1h/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/ES/1h/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/R

...

/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_csv.iloc[:, 1:3] = ''
/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_csv.iloc[:, 1:3] = ''
/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_csv.iloc[:, 1:3] = ''
/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible

Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/PL/1h/2023.csv'.


/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_csv.iloc[:, 1:3] = ''
/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_csv.iloc[:, 1:3] = ''
/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_csv.iloc[:, 1:3] = ''
/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible

Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/PT/1h/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/RO/1h/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/RO/30min/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/RO/15min/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/SE/1h/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/SI/1h/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/SK/1h/2023.csv'.
Values erased and first column renamed in CSV file '/home/ray/Dispa-SET_Unleash/RawD

/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_csv.iloc[:, 1:3] = ''
/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_csv.iloc[:, 1:3] = ''
/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_csv.iloc[:, 1:3] = ''
/tmp/ipykernel_69186/1195302579.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
7.2. Reservoirs Level Clean Files Interpolation
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Completing the time data to the clean Reservoirs Level files.
<br>
For interpolation purposes it is needed to add new dates from the next year to the corresponding Reservoirs Level Clean files.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Additionally is needed remove duplicated dates if there are any.
</div>
<br>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Completing the dates to interpolation purposes.
<br>

In [37]:
def add_new_rows(time_step):
    # Define reference data file path
    # reference_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Reference_Data.csv"

    # Read the reference data CSV file
    df_reference = pd.read_csv(reference_data_file_path)

    # Iterate over each row in the DataFrame
    for index, row in df_reference.iterrows():
        # Get the file paths from the specified columns
        file_path_1 = row[time_step]
        file_path_2 = row['Single_Raw_Data_File_Path']

        # Check if file_path_1 is not NaN
        if isinstance(file_path_1, str):  # Check if file_path_1 is a string
            # Read the first column (time) from file_path_1
            df1 = pd.read_csv(file_path_1)
            last_date_file1 = pd.to_datetime(df1.iloc[-1, 0])

            # Read the 'Dispa-SET_Date' column from file_path_2
            df2 = pd.read_csv(file_path_2)
            last_date_file2 = pd.to_datetime(df2['Dispa-SET_Date'].iloc[-1])

            # Check if the last date from file_path_1 is less than the last date from file_path_2
            if last_date_file1 < last_date_file2:
                # Calculate time steps until reaching the last date from file_path_2
                if time_step == '1h':
                    freq = '1H'
                elif time_step == '30min':
                    freq = '30Min'
                elif time_step == '15min':
                    freq = '15Min'
                else:
                    print("Invalid time step frequency.")
                    return
                
                time_steps = pd.date_range(start=last_date_file1, end=last_date_file2, freq=freq)

                # Create a new DataFrame with time steps, only in the first column
                df_new_rows = pd.DataFrame({df1.columns[0]: time_steps[1:]})

                # Append the new rows to the first column of file_path_1 DataFrame
                df1 = pd.concat([df1, df_new_rows], ignore_index=True)

                # Save the updated DataFrame back to the original CSV file
                df1.to_csv(file_path_1, index=False)
                print(f"New rows added successfully to {file_path_1}")
            else:
                print(f"No new rows need to be added to {file_path_1}")
        else:
            print(f"Invalid file path for {time_step} in row {index}: NaN")

    print("Overall processing finished.")

# Add new rows with 1 hour time step
add_new_rows('1h')

# Add new rows with 30 minute time step
add_new_rows('30min')

# Add new rows with 15 minute time step
add_new_rows('15min')

/tmp/ipykernel_69186/121062958.py:37: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_steps = pd.date_range(start=last_date_file1, end=last_date_file2, freq=freq)
/tmp/ipykernel_69186/121062958.py:37: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_steps = pd.date_range(start=last_date_file1, end=last_date_file2, freq=freq)


New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/1h/2023.csv
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/1h/2023.csv
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/1h/2023.csv


/tmp/ipykernel_69186/121062958.py:37: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_steps = pd.date_range(start=last_date_file1, end=last_date_file2, freq=freq)
/tmp/ipykernel_69186/121062958.py:37: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_steps = pd.date_range(start=last_date_file1, end=last_date_file2, freq=freq)


New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/1h/2023.csv
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/1h/2023.csv
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CZ/1h/2023.csv
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DE/1h/2023.csv
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DK/1h/2023.csv
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EE/1h/2023.csv
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EL/1h/2023.csv
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/ES/1h/2023.csv
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/FI/1h/2023.csv
New rows added successfully to /home/

/tmp/ipykernel_69186/121062958.py:37: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_steps = pd.date_range(start=last_date_file1, end=last_date_file2, freq=freq)
/tmp/ipykernel_69186/121062958.py:37: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_steps = pd.date_range(start=last_date_file1, end=last_date_file2, freq=freq)
/tmp/ipykernel_69186/121062958.py:37: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_steps = pd.date_range(start=last_date_file1, end=last_date_file2, freq=freq)
/tmp/ipykernel_69186/121062958.py:37: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_steps = pd.date_range(start=last_date_file1, end=last_date_file2, freq=freq)
/tmp/ipykernel_69186/121062958.py:37: FutureWarning: 'H' is deprecated and will be removed in a future version, please u

...

New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/SE/1h/2023.csv


/tmp/ipykernel_69186/121062958.py:37: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_steps = pd.date_range(start=last_date_file1, end=last_date_file2, freq=freq)


New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/SI/1h/2023.csv
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/SK/1h/2023.csv
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/UK/1h/2023.csv
Overall processing finished.
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/30min/2023.csv
Invalid file path for 30min in row 1: NaN
Invalid file path for 30min in row 2: NaN
Invalid file path for 30min in row 3: NaN
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/30min/2023.csv
Invalid file path for 30min in row 5: NaN


/tmp/ipykernel_69186/121062958.py:37: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_steps = pd.date_range(start=last_date_file1, end=last_date_file2, freq=freq)
/tmp/ipykernel_69186/121062958.py:37: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_steps = pd.date_range(start=last_date_file1, end=last_date_file2, freq=freq)


New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DE/30min/2023.csv
Invalid file path for 30min in row 7: NaN
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/EE/30min/2023.csv
Invalid file path for 30min in row 9: NaN
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/ES/30min/2023.csv
Invalid file path for 30min in row 11: NaN
Invalid file path for 30min in row 12: NaN
Invalid file path for 30min in row 13: NaN
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/HU/30min/2023.csv
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/IE/30min/2023.csv
Invalid file path for 30min in row 16: NaN
Invalid file path for 30min in row 17: NaN
New rows added successfully to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/LU/30min/2023.csv
Invalid file path for 30min in row 19: Na

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Erasing duplicated dates.
<br>

In [38]:
# Define reference data file path
#reference_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/Reference_Data.csv"

# Read the reference data CSV file
df_reference = pd.read_csv(reference_data_file_path)

# Iterate over each row in the DataFrame
for index, row in df_reference.iterrows():
    # Get the file paths from the specified columns
    file_paths = {'1h': row['1h'], '30min': row['30min'], '15min': row['15min']}

    for time_step, file_path in file_paths.items():
        # Check if file_path is not NaN and is a string
        if isinstance(file_path, str):  
            # Read the CSV file
            df = pd.read_csv(file_path)

            # Remove rows with duplicated values in the first column
            if not df.empty:  # Check if the DataFrame is not empty
                df = df.drop_duplicates(subset=df.columns[0], keep='first')

                # Save the updated DataFrame back to the original CSV file
                df.to_csv(file_path, index=False)

                print(f"Duplicates removed successfully from {time_step} file: {file_path}")
            else:
                print(f"Empty DataFrame for {time_step} file: {file_path}. No duplicates to remove.")
        else:
            print(f"Invalid file path for {time_step} in row {index}: NaN")

print("Overall processing finished.")

Duplicates removed successfully from 1h file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/1h/2023.csv
Duplicates removed successfully from 30min file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/30min/2023.csv
Duplicates removed successfully from 15min file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/15min/2023.csv
Duplicates removed successfully from 1h file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/1h/2023.csv
Invalid file path for 30min in row 1: NaN
Invalid file path for 15min in row 1: NaN
Duplicates removed successfully from 1h file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/1h/2023.csv
Invalid file path for 30min in row 2: NaN
Invalid file path for 15min in row 2: NaN
Duplicates removed successfully from 1h file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/1h/2023.csv
Invalid file path for 30min in row 3: NaN
Invalid file path for 15min in row 3: NaN
Dupl

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Copying the weekly data to the clean csv Reservoirs Level files.
<div/>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
For interpolation purposes it is needed copy the data per week to the corresponding date in the clean csv Reservoirs Level files.
<div/>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
This is done for all the 30 countries modelled in Dispa-SET and for each corresponding time step as well (1 hou, 30 minutes and 15 minutes)
</div>

In [39]:
# Read the reference data CSV file
df_reference = pd.read_csv(reference_data_file_path)

# Iterate over each row in the DataFrame
for index, row in df_reference.iterrows():
    for time_interval in ['1h', '30min', '15min']:
        # Get the file paths from the specified columns
        file_path_1 = row[time_interval]
        file_path_2 = row['Single_Raw_Data_File_Path']

        # Check if both file paths are not NaN
        if isinstance(file_path_1, str) and isinstance(file_path_2, str):
            # Read CSV files
            df_1 = pd.read_csv(file_path_1)
            df_2 = pd.read_csv(file_path_2)

            # Get the data year from the file path
            data_year = int(file_path_1.split('/')[-1].split('.')[0])

            # Get the column name corresponding to the data year
            year_column = str(data_year)

            # Iterate over each row in df_2
            for _, row_2 in df_2.iterrows():
                # Get the value from the 'Dispa-SET_Date' column in df_2
                date_value = row_2['Dispa-SET_Date']

                # Check if the value exists in the first column of df_1
                if date_value in df_1[df_1.columns[0]].values:
                    # Get the index where the value is found in df_1
                    index_value = df_1.index[df_1[df_1.columns[0]] == date_value][0]
                    # Copy corresponding value from df_2 and paste it to the corresponding fields of the second and third columns in df_1
                    df_1.at[index_value, df_1.columns[1]] = row_2[year_column]
                    df_1.at[index_value, df_1.columns[2]] = row_2[year_column]

            # Save the updated DataFrame back to the original CSV file
            df_1.to_csv(file_path_1, index=False)
            print(f"Values copied successfully from {file_path_2} to {file_path_1}")

        else:
            print(f"Invalid file paths in row {index}: NaN")

print("Overall processing finished.")

Values copied successfully from /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/2023_1.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/1h/2023.csv
Values copied successfully from /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/2023_1.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/30min/2023.csv
Values copied successfully from /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/2023_1.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/15min/2023.csv
Values copied successfully from /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/2023_1.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/1h/2023.csv
Invalid file paths in row 1: NaN
Invalid file paths in row 1: NaN
Values copied successfully from /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/2023_1.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/1h/2023.cs

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Interpolating the clean csv Reservoirs Level fiels.
<div/>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
A linear interpolation process is used to get the values needed between each weekly data.
<br>
This is done for all the 30 countries modelled in Dispa-SET and for each corresponding time step as well (1 hou, 30 minutes and 15 minutes)
</div>

In [40]:
# Read the reference data CSV file
df_reference = pd.read_csv(reference_data_file_path)

# Define the columns to be interpolated
columns_to_interpolate = ['HPHS', 'HDAM']  # Replace with actual column names

# Iterate over each row in the DataFrame
for index, row in df_reference.iterrows():
    # Get the file paths from the specified columns
    file_path_1h = row['1h']
    file_path_30min = row['30min']
    file_path_15min = row['15min']

    # Check if any of the file paths are not NaN
    if isinstance(file_path_1h, str):
        # Read CSV file for 1h
        df_1h = pd.read_csv(file_path_1h)
        # Perform linear interpolation for 1h columns
        df_1h[columns_to_interpolate] = df_1h[columns_to_interpolate].interpolate(method='linear')
        # Save the interpolated DataFrame back to the CSV file
        df_1h.to_csv(file_path_1h, index=False)
        print(f"Linear interpolation completed for {file_path_1h}")

    if isinstance(file_path_30min, str):
        # Read CSV file for 30min
        df_30min = pd.read_csv(file_path_30min)
        # Perform linear interpolation for 30min columns
        df_30min[columns_to_interpolate] = df_30min[columns_to_interpolate].interpolate(method='linear')
        # Save the interpolated DataFrame back to the CSV file
        df_30min.to_csv(file_path_30min, index=False)
        print(f"Linear interpolation completed for {file_path_30min}")

    if isinstance(file_path_15min, str):
        # Read CSV file for 15min
        df_15min = pd.read_csv(file_path_15min)
        # Perform linear interpolation for 15min columns
        df_15min[columns_to_interpolate] = df_15min[columns_to_interpolate].interpolate(method='linear')
        # Save the interpolated DataFrame back to the CSV file
        df_15min.to_csv(file_path_15min, index=False)
        print(f"Linear interpolation completed for {file_path_15min}")

print("Overall processing finished.")

Linear interpolation completed for /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/1h/2023.csv
Linear interpolation completed for /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/30min/2023.csv
Linear interpolation completed for /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/15min/2023.csv
Linear interpolation completed for /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/1h/2023.csv
Linear interpolation completed for /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/1h/2023.csv
Linear interpolation completed for /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/1h/2023.csv
Linear interpolation completed for /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/1h/2023.csv
Linear interpolation completed for /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/30min/2023.csv
Linear interpolation completed for /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CZ/1h/2

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Getting the final Reservoirs Level file.
<div/>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
A process of dates filtering is done, since just the analized year (specified in the data_year variable) is required.
<br>
This is done for all the 30 countries modelled in Dispa-SET and for each corresponding time step as well (1 hou, 30 minutes and 15 minutes)
</div>

In [41]:
# Define a function to process each file
def process_file(file_path):
    if not os.path.isfile(file_path):
        print(f"No file found at {file_path}")
        return None

    # Read the CSV file
    df = pd.read_csv(file_path)

    # Convert the first column to datetime
    df['Dispa-SET_Date'] = pd.to_datetime(df['Dispa-SET_Date'])

    # Extract the year from the date values
    df['Year'] = df['Dispa-SET_Date'].dt.year

    # Filter out rows that belong to the specified year
    data_year = int(os.path.basename(file_path).split('.')[0])
    df_filtered = df[df['Year'] == data_year].copy()  # Create a copy to avoid SettingWithCopyWarning

    # Drop the 'Year' column
    df_filtered.drop(columns=['Year'], inplace=True)

    return df_filtered

# Read the reference data CSV file
df_reference = pd.read_csv(reference_data_file_path)

# Iterate over each row in the DataFrame
for index, row in df_reference.iterrows():
    # Get the file paths from the specified columns
    file_path_1h = row['1h']
    file_path_30min = row['30min']
    file_path_15min = row['15min']

    # Process the file path for '1h'
    if isinstance(file_path_1h, str):
        df_filtered_1h = process_file(file_path_1h)
        if df_filtered_1h is not None:
            df_filtered_1h.to_csv(file_path_1h, index=False)
            print(f"Rows not belonging to {data_year} have been removed from {file_path_1h}")
        else:
            print(f"No file found for '1h' at index {index}")

    # Process the file path for '30min'
    if isinstance(file_path_30min, str):
        df_filtered_30min = process_file(file_path_30min)
        if df_filtered_30min is not None:
            df_filtered_30min.to_csv(file_path_30min, index=False)
            print(f"Rows not belonging to {data_year} have been removed from {file_path_30min}")
        else:
            print(f"No file found for '30min' at index {index}")

    # Process the file path for '15min'
    if isinstance(file_path_15min, str):
        df_filtered_15min = process_file(file_path_15min)
        if df_filtered_15min is not None:
            df_filtered_15min.to_csv(file_path_15min, index=False)
            print(f"Rows not belonging to {data_year} have been removed from {file_path_15min}")
        else:
            print(f"No file found for '15min' at index {index}")

print("Overall processing finished.")

Rows not belonging to 2023 have been removed from /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/1h/2023.csv
Rows not belonging to 2023 have been removed from /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/30min/2023.csv
Rows not belonging to 2023 have been removed from /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/15min/2023.csv
Rows not belonging to 2023 have been removed from /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/1h/2023.csv
Rows not belonging to 2023 have been removed from /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/1h/2023.csv
Rows not belonging to 2023 have been removed from /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/1h/2023.csv
Rows not belonging to 2023 have been removed from /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/1h/2023.csv
Rows not belonging to 2023 have been removed from /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/C

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Updating the Dispa-SET time column.
<div/>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Since this column header does not have to have any value, the same is erased.
</div>

In [42]:
def rename_column(reference_data_file_path):
    """Renames the first column of specified CSV files.

    Args:
        reference_data_file_path (str): Path to the reference data CSV file.
    """

    df_reference = pd.read_csv(reference_data_file_path)

    for index, row in df_reference.iterrows():
        for time_interval in ['1h', '30min', '15min']:
            file_path = row[time_interval]

            if pd.notnull(file_path):  # Check if file path is not null
                df = pd.read_csv(file_path)
                df.columns = ['', df.columns[1], df.columns[2]]
                df.to_csv(file_path, index=False)
                print(f"Renamed first column in {file_path}")

rename_column(reference_data_file_path)

Renamed first column in /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/1h/2023.csv
Renamed first column in /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/30min/2023.csv
Renamed first column in /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/15min/2023.csv
Renamed first column in /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/1h/2023.csv
Renamed first column in /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/1h/2023.csv
Renamed first column in /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/1h/2023.csv
Renamed first column in /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/1h/2023.csv
Renamed first column in /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/30min/2023.csv
Renamed first column in /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CZ/1h/2023.csv
Renamed first column in /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DE/1h/

<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Erasing empty subfolders.
<div/>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
For the next stept the already processed data is going to be copied to the Dispa-SET data base.
<br>
Empty folder are not necesary to be copied, so they are erased.
</div>

In [43]:
def remove_empty_subfolders(reference_data_file_path):
    """Removes empty subfolders within specified folders.

    Args:
        reference_data_file_path (str): Path to the reference data CSV file.
    """

    df_reference = pd.read_csv(reference_data_file_path)

    for index, row in df_reference.iterrows():
        zone_folder = row['Zone_Folder_Path']

        for time_interval in ['1h', '30min', '15min']:
            subfolder_path = os.path.join(zone_folder, time_interval)

            if os.path.exists(subfolder_path) and not os.listdir(subfolder_path):
                os.rmdir(subfolder_path)
                print(f"Removed empty subfolder: {subfolder_path}")

remove_empty_subfolders(reference_data_file_path)

Removed empty subfolder: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/30min
Removed empty subfolder: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/15min
Removed empty subfolder: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/30min
Removed empty subfolder: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/15min
Removed empty subfolder: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/30min
Removed empty subfolder: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/15min
Removed empty subfolder: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CY/15min
Removed empty subfolder: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CZ/30min
Removed empty subfolder: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CZ/15min
Removed empty subfolder: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/DK/30min
Removed empty subfolder: /home/ray/Dispa-SET_Unlea

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
8. Reservoir Level Files
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
The already processed Reservoirs Level Time Series are copied to the Dispa-SET data base.
<br>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Defining the neede directory paths.
</div>

In [44]:
# Additional string to be appended
additional_path_4 = "/Database/HydroData/ReservoirLevel/"

# Construct the reservoir_level_dispa-set_folder_path variable
reservoir_level_dispaset_folder_path = dispaSET_unleash_folder_path + additional_path_4

print("reservoir_level_dispaset_folder_path:", reservoir_level_dispaset_folder_path)

reservoir_level_dispaset_folder_path: /home/ray/Dispa-SET_Unleash/Database/HydroData/ReservoirLevel/


<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Copying the final reservoir level files.
<br>

In [45]:
# Read the reference data CSV file
df_reference = pd.read_csv(reference_data_file_path)

# Iterate over each row in the DataFrame
for index, row in df_reference.iterrows():
    zone_folder_path = row['Zone_Folder_Path']

    # Iterate over each time resolution subfolder ('1h', '15min', '30min')
    for time_resolution in ['1h', '15min', '30min']:
        source_file_path = os.path.join(zone_folder_path, time_resolution, f"{data_year}.csv")
        
        # Check if the source file exists
        if os.path.isfile(source_file_path):
            dispaSET_code = os.path.basename(zone_folder_path)  # Extract the folder name, assumed to be the Dispa-SET code
            destination_folder = os.path.join(reservoir_level_dispaset_folder_path, dispaSET_code, time_resolution)

            # Create the destination directory if it doesn't exist
            os.makedirs(destination_folder, exist_ok=True)
            
            # Define the destination file path
            destination_file_path = os.path.join(destination_folder, f"{data_year}.csv")
            
            # Copy the file
            shutil.copy2(source_file_path, destination_file_path)
            print(f"Copied {source_file_path} to {destination_file_path}")
        else:
            print(f"File not found: {source_file_path}")

print("All files processed.")

Copied /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/1h/2023.csv to /home/ray/Dispa-SET_Unleash/Database/HydroData/ReservoirLevel/AT/1h/2023.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/15min/2023.csv to /home/ray/Dispa-SET_Unleash/Database/HydroData/ReservoirLevel/AT/15min/2023.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/30min/2023.csv to /home/ray/Dispa-SET_Unleash/Database/HydroData/ReservoirLevel/AT/30min/2023.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/1h/2023.csv to /home/ray/Dispa-SET_Unleash/Database/HydroData/ReservoirLevel/BE/1h/2023.csv
File not found: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/15min/2023.csv
File not found: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/30min/2023.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/1h/2023.csv to /home/ray/Dispa-SET_Unleash/Database/HydroData/ReservoirLeve

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
9. Scaled Inflows Data
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
The already processed Reservoirs Level Time Series are needed to be used to process the Scaled Inflows time series.
<br>
The next cells are use to get the apropiate data and then copy it to the sacled inflow directory
<br>
<br>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Defining the needed directory paths.
</div>

In [46]:
additional_path_6 = "/RawData/HydroData/ScaledInflows/Levels/"

# Construct the_level_folder_path variable
level_folder_path = dispaSET_unleash_folder_path + additional_path_6

print("level_folder_path:", level_folder_path)

level_folder_path: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Levels/


<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Copyin the needed time series to the level folder in the scaled inflows directory.
</div>

In [47]:
def copy_csv_files(dispaSET_codes, reservoir_level_folder_path, level_folder_path, data_year):
  """
  Copies CSV files from subfolders to a new structure.

  Args:
    dispaSET_codes: A list of dispaSET codes.
    reservoir_level_folder_path: The path to the source folder.
    level_folder_path: The path to the destination folder.
    data_year: The year for the CSV files.
  """

  for code in dispaSET_codes:
    source_code_folder = os.path.join(reservoir_level_folder_path, code)
    destination_code_folder = os.path.join(level_folder_path, code)

    if not os.path.exists(destination_code_folder):
      os.makedirs(destination_code_folder)

    for subfolder in ['1h', '30min', '15min']:
      source_file = os.path.join(source_code_folder, subfolder, f"{data_year}.csv")
      destination_file = os.path.join(destination_code_folder, subfolder, f"{data_year}.csv")

      if os.path.exists(source_file):
        os.makedirs(os.path.dirname(destination_file), exist_ok=True)
        shutil.copy2(source_file, destination_file)
        print(f"Copied {source_file} to {destination_file}")

copy_csv_files(dispaSET_codes, reservoir_level_folder_path, level_folder_path, data_year)

Copied /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/1h/2023.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Levels/AT/1h/2023.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/30min/2023.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Levels/AT/30min/2023.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/AT/15min/2023.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Levels/AT/15min/2023.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BE/1h/2023.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Levels/BE/1h/2023.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/BG/1h/2023.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Levels/BG/1h/2023.csv
Copied /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel/CH/1h/2023.csv to /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Le

<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Adding two new columns to get the value of the levels in MW.
</div>

In [48]:
def multiply_columns(data_year, dispaSET_codes, level_folder_path, reference_data_file_path):
    """Multiply the HPHS and HDAM columns in the CSV files by the corresponding Maximum_Level value.
       Save the results in new columns HPHS_MW and HDAM_MW."""
    
    # Read the reference data file
    df_reference = pd.read_csv(reference_data_file_path)
    
    for dispa_code in dispaSET_codes:
        # Find the corresponding Maximum_Level for the current dispaSET_code
        max_level = df_reference.loc[df_reference['Dispa-SET_Code'] == dispa_code, 'Maximum_Level'].values[0]
        
        # Construct the path to the folder for this dispaSET_code
        dispa_folder_path = os.path.join(level_folder_path, dispa_code)
        
        # List of subfolders to check (1h, 30min, 15min)
        subfolders = ['1h', '30min', '15min']
        
        for subfolder in subfolders:
            subfolder_path = os.path.join(dispa_folder_path, subfolder)
            csv_file_path = os.path.join(subfolder_path, f"{data_year}.csv")
            
            # Check if the CSV file exists
            if os.path.exists(csv_file_path):
                # Read the CSV file
                df = pd.read_csv(csv_file_path)
                
                # Multiply HPHS and HDAM columns by max_level and create new columns
                df['HPHS_MW'] = df['HPHS'] * max_level
                df['HDAM_MW'] = df['HDAM'] * max_level
                
                # Save the updated CSV file with the new columns
                df.to_csv(csv_file_path, index=False)
                
                # Print a message indicating that the file was processed
                print(f"Processed file: {csv_file_path}")
            else:
                print(f"File not found: {csv_file_path}")

multiply_columns(data_year, dispaSET_codes, level_folder_path, reference_data_file_path)

Processed file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Levels/AT/1h/2023.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Levels/AT/30min/2023.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Levels/AT/15min/2023.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Levels/BE/1h/2023.csv
File not found: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Levels/BE/30min/2023.csv
File not found: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Levels/BE/15min/2023.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Levels/BG/1h/2023.csv
File not found: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Levels/BG/30min/2023.csv
File not found: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Levels/BG/15min/2023.csv
Processed file: /home/ray/Dispa-SET_Unleash/RawData/HydroData/ScaledInflows/Levels/CH/1h/2023.csv
Fi

<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman; color:skyblue">
9. Reservoir Level Folder Back Up
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Once all the formating process was done the Reservoir Level Folder is restored to its defoult state.
</div>

In [49]:
if os.path.exists(reservoir_level_folder_path):
    shutil.rmtree(reservoir_level_folder_path)  # Remove the current directory
shutil.copytree(backup_folder_path, reservoir_level_folder_path)

print(f"Directory restored to original state from {backup_folder_path}")

Directory restored to original state from /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel_backup/


In [50]:
shutil.rmtree(backup_folder_path)
print(f"Backup folder {backup_folder_path} deleted successfully.")

Backup folder /home/ray/Dispa-SET_Unleash/RawData/HydroData/ReservoirLevel_backup/ deleted successfully.
